In [1]:
import torch
import torch.nn.functional as F
import torchvision
import torch.nn as nn
from torchvision import transforms as T
from utils import data, evaluate, models, train, plot
from torchsummary import summary
from torchvision.models import googlenet, GoogLeNet_Weights

# Parameter Initialization

In [2]:
data_path = 'data/asl_alphabet_train'
data_path_test_1 = 'data/asl_alphabet_test'
data_path_test_2 = 'data/extra_test'
samples = 87000
split_factor = [0.8, 0.2, 0.0]
batch_size = 128
lr = 5e-4
epochs = 2
image_size = (100, 100)
l2_reg = 1e-4

# Get Train, Val and Test Split

In [3]:
torch.manual_seed(0)
train_trans = T.Compose([T.ToTensor(), T.Resize(image_size), T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1), 
                         T.RandomRotation(degrees=(-5,5)), T.RandomHorizontalFlip(0.2), 
                         T.Normalize((0.5190, 0.4992, 0.5140), (0.2244, 0.2523, 0.2616))])
test_trans = T.Compose([T.ToTensor(), T.Resize(image_size), T.Normalize((0.5190, 0.4992, 0.5140), (0.2244, 0.2523, 0.2616))])

dataset = torchvision.datasets.ImageFolder(root=data_path, transform=train_trans)
test_dataset_1 = torchvision.datasets.ImageFolder(root=data_path_test_1, transform=test_trans)
test_dataset_2 = torchvision.datasets.ImageFolder(root=data_path_test_2, transform=test_trans)

train_loader, valid_loader = data.split_dataset(dataset, samples, split_factor, batch_size)
test_loader_1 = torch.utils.data.DataLoader(test_dataset_1, batch_size=batch_size, shuffle=False)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=batch_size, shuffle=False)

# Model

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')

model = torchvision.models.googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
model = model.to(device)
summary(model, input_size=(3, image_size[0], image_size[1]))

Using cuda device
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 50, 50]           9,408
       BatchNorm2d-2           [-1, 64, 50, 50]             128
       BasicConv2d-3           [-1, 64, 50, 50]               0
         MaxPool2d-4           [-1, 64, 25, 25]               0
            Conv2d-5           [-1, 64, 25, 25]           4,096
       BatchNorm2d-6           [-1, 64, 25, 25]             128
       BasicConv2d-7           [-1, 64, 25, 25]               0
            Conv2d-8          [-1, 192, 25, 25]         110,592
       BatchNorm2d-9          [-1, 192, 25, 25]             384
      BasicConv2d-10          [-1, 192, 25, 25]               0
        MaxPool2d-11          [-1, 192, 12, 12]               0
           Conv2d-12           [-1, 64, 12, 12]          12,288
      BatchNorm2d-13           [-1, 64, 12, 12]             128
      BasicConv2d-14 

## 1) Training Phase

In [5]:
train_loss_list, train_accuracy_list, val_loss_list, val_accuracy_list = train.train_model(model, train_loader, valid_loader, 
                                                                                           lr, epochs, device, l2_reg)

Epoch 1 -> Loss = 0.0153 | Train Accuracy = 99.52% | valation Accuracy = 99.45%
Epoch 2 -> Loss = 0.0359 | Train Accuracy = 98.92% | valation Accuracy = 99.08%
----------------------------------------------------------------------------------------------------
Time taken to train: 2172.58s
Average time of each epoch: 1086.29s


## 2) Testing Phase

### i) Test Set 1 Accuracy

In [6]:
test_accuracy, y_true, y_pred = evaluate.evaluate_model(model, test_loader_1, device)

Test Accuracy = 100.0000%
F1 Score = 1.0000
Recall = 1.0000
Precision = 1.0000


### i) Test Set 2 Accuracy

In [7]:
test_accuracy, y_true, y_pred = evaluate.evaluate_model(model, test_loader_2, device)

Test Accuracy = 43.5632%
F1 Score = 0.4158
Recall = 0.4356
Precision = 0.4839


C:\Users\rohit\anaconda3\envs\ee541\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
